In [2]:
from functools import reduce
import datetime
import csv
import os


def menu_principal():
    menu = """Digite o número da opção desejada:

    1 - Cadastro de Produto
    2 - Atualização de Produto
    3 - Exclusão de Produto
    4 - Listagem de Produtos
    5 - Venda de Produto
    6 - Gerar Relatório
    7 - Finalizar o Sistema
    Escolha: """
    while True:
        try:
            opcao = int(input(menu))
            if opcao not in range(1, 8):
                raise ValueError("Opção inválida! Tente novamente.")
            return opcao
        except ValueError as e:
            print(e)

def exibir_boas_vindas():
    mensagem = """
    ========================================
    =             Bem-vindo ao             =
    =  Gerenciador de Inventário AdaEletro =
    ========================================
    """
    return print(mensagem)

def exibir_menu_principal():
    mensagem = """
    ========================================
    =            Menu Principal            =
    ========================================
    """
    return print(mensagem)


def cadastrar_produto(produtos_cadastrados, proximo_id):
    try:
        nome = input("\nDigite o nome do produto: ")

        if checar_produto(produtos_cadastrados, nome):
            print("Produto já cadastrado!")
            return

        categoria = input("Digite a categoria do produto: ")
        preco = float(input("Digite o preço do produto: "))
        quantidade = int(input("Digite a quantidade em estoque do produto: "))
        descricao = input("Digite a descrição do produto (opcional): ")

        produto = {
            'id': proximo_id,
            'nome': nome,
            'categoria': categoria,
            'preco': preco,
            'quantidade': quantidade,
            'descricao': descricao
        }

        produtos_cadastrados.append(produto)
        proximo_id += 1

        print("Produto cadastrado com sucesso!")
        return proximo_id
    except ValueError:
        print("Dado(s) inválido(s)!")
        return proximo_id

def checar_produto(produtos_cadastrados, nome):
    produto_encontrado = list(filter(lambda p: p['nome'] == nome, produtos_cadastrados))
    return len(produto_encontrado) > 0

def atualizar_produto(produtos_cadastrados):
    try:
        produto_id = int(input("Digite o ID do produto a ser atualizado: "))

        produto = None
        for p in produtos_cadastrados:
            if p['id'] == produto_id:
                produto = p
                break

        if produto is None:
            print("Produto não encontrado!")
            return

        print("\nInformações atuais do produto:")
        print(f"ID: {produto_id}")
        print(f"Nome: {produto['nome']}")
        print(f"Categoria: {produto['categoria']}")
        print(f"Preço: {produto['preco']}")
        print(f"Quantidade em estoque: {produto['quantidade']}")
        print(f"Descrição: {produto['descricao']}")

        novo_preco = input("Digite o novo preço do produto (ou pressione Enter para manter o valor atual): ")
        nova_categoria = input("Digite a nova categoria do produto (ou pressione Enter para manter a categoria atual): ")
        nova_quantidade = input("Digite a nova quantidade em estoque do produto (ou pressione Enter para manter o valor atual): ")
        nova_descricao = input("Digite a nova descrição do produto (ou pressione Enter para manter a descrição atual): ")

        if novo_preco:
            novo_preco = float(novo_preco)
            if novo_preco < 0:
                raise ValueError("Preço inválido! Deve ser um valor positivo.")
            produto['preco'] = novo_preco

        if nova_categoria:
            produto['categoria'] = nova_categoria

        if nova_quantidade:
            nova_quantidade = int(nova_quantidade)
            if nova_quantidade < 0:
                raise ValueError("Quantidade inválida! Deve ser um valor positivo.")
            produto['quantidade'] = nova_quantidade

        if nova_descricao:
            produto['descricao'] = nova_descricao

        print("\nProduto atualizado com sucesso!")
    except ValueError as e:
        print(f"Dado(s) inválido(s): {e}\nCancelando atualização de dados!")

def excluir_produto(produtos_cadastrados):
    try:
        produto_id = int(input("Digite o ID do produto: "))

        for i, produto in enumerate(produtos_cadastrados):
            if produto['id'] == produto_id:
                del produtos_cadastrados[i]
                print("Produto excluído com sucesso!")
                return

        print("Produto não encontrado!")
    except ValueError:
        print("Dado(s) inválido(s)!")

def realizar_venda(produtos_cadastrados, **produtos_vendidos):
    agora = datetime.datetime.now()
    data_e_hora_formatada = agora.strftime("%d/%m/%Y %H:%M:%S")

    vendas_realizadas = []

    for nome, quantidade in produtos_vendidos.items():
        produto_encontrado = False

        for produto in produtos_cadastrados:
            if produto['nome'] == nome:
                produto_encontrado = True
                if quantidade <= produto['quantidade']:
                    produto['quantidade'] -= quantidade
                    vendas_realizadas.append({'nome': nome, 'quantidade': quantidade, 'preco': produto['preco'], 'preco_total': quantidade * produto['preco'],'data_e_hora': data_e_hora_formatada})
                    print(f"{quantidade} unidade(s) do produto '{nome}' vendida(s) com sucesso.")
                else:
                    print(f"A quantidade informada ({quantidade}) excede o estoque disponível do produto '{nome}'.")
                    sugerir_alternativas(produtos_cadastrados, nome, produto['categoria'])
                break

        if not produto_encontrado:
            print(f"O produto '{nome}' não foi encontrado no sistema.")

    total_vendas = reduce(lambda valor, produto: valor + produto['preco_total'], vendas_realizadas, 0)
    print(f"Total da venda: R$ {total_vendas:.2f}")

    return vendas_realizadas

def sugerir_alternativas(produtos_cadastrados, nome, categoria):
    alternativas = list(filter(lambda produto : produto['categoria'] == categoria and produto['quantidade'] > 0 and produto['nome'] != nome, produtos_cadastrados))

    if alternativas:
        print("\nSugestões de produtos alternativos na mesma categoria:")
        for produto in alternativas:
            print(f"ID: {produto['id']} | Nome: {produto['nome']} | Quantidade: {produto['quantidade']}")
    else:
        print("Nenhum produto alternativo disponível.")


def gerar_relatorio(vendas_realizadas):
    try:
        file_exists = os.path.isfile('Relatorio_Vendas.csv')
        with open('Relatorio_Vendas.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            if not file_exists or os.stat('Relatorio_Vendas.csv').st_size == 0:
                writer.writerow(['Nome do Produto', 'Quantidade Vendida', 'Preço Unitário', 'Valor Total', 'Data/Hora da Venda'])
            for venda in vendas_realizadas:
                writer.writerow([venda['nome'], venda['quantidade'], venda['preco'], venda['preco_total'], venda['data_e_hora']])
        print("Relatório de vendas gerado com sucesso!")
    except IOError:
        print("Erro ao gerar relatório de vendas!")


def listar_produtos(produtos_cadastrados):
    if not produtos_cadastrados:
        print("\nNenhum produto cadastrado.")
    else:
        print("\nLista de Produtos:")
        print("ID | Nome | Categoria | Preço | Quantidade | Descrição")
        print("=" * 65)
        for produto in produtos_cadastrados:
            print(f"{produto['id']} | {produto['nome']} | {produto['categoria']} | {produto['preco']} | {produto['quantidade']} | {produto['descricao']}")
        print("=" * 65)

def sistema():
    estoque_produtos = []
    file = open('Base trabalho final.csv', mode='r', encoding='latin1')
    csv_reader = csv.DictReader(file, delimiter=';')

    try:
        for row in csv_reader:
            produto = {
                'id': int(row['ID']),
                'nome': row['Nome'],
                'categoria': row['Categoria'],
                'preco': float(row['Preço']),
                'quantidade': int(row['Quantidade em Estoque']),
                'descricao': row['Descrição']
            }
            estoque_produtos.append(produto)
    finally:
      file.close()

    tamanho_produto = len(estoque_produtos)
    return estoque_produtos, tamanho_produto + 1

def atualizar_csv_estoque(produtos_cadastrados):
    try:
        with open('Base trabalho final.csv', mode='w', newline='', encoding='latin1') as file:
            campos = ['ID', 'Nome', 'Categoria', 'Preço', 'Quantidade em Estoque', 'Descrição']
            writer = csv.DictWriter(file, fieldnames=campos, delimiter=';')
            writer.writeheader()
            for produto in produtos_cadastrados:
                writer.writerow({
                    'ID': produto['id'],
                    'Nome': produto['nome'],
                    'Categoria': produto['categoria'],
                    'Preço': produto['preco'],
                    'Quantidade em Estoque': produto['quantidade'],
                    'Descrição': produto['descricao']
                })
        print("Estoque atualizado e salvo com sucesso!")
    except IOError:
        print("Erro ao atualizar o arquivo csv!")

def main():
    produtos_cadastrados, proximo_id = sistema()
    exibir_boas_vindas()
    vendas_realizadas = []
    while True:
        exibir_menu_principal()
        escolha = menu_principal()
        if escolha == 1:
            proximo_id = cadastrar_produto(produtos_cadastrados, proximo_id)
        elif escolha == 2:
            atualizar_produto(produtos_cadastrados)
        elif escolha == 3:
            excluir_produto(produtos_cadastrados)
        elif escolha == 4:
            listar_produtos(produtos_cadastrados)
        elif escolha == 5:
            produtos_vendidos = {}
            while True:
                nome = input("Digite o nome do produto a ser vendido ou Enter para finalizar: ")
                if not nome:
                    break
                while True:
                    try:
                        quantidade = int(input("Digite a quantidade do produto: "))
                        break
                    except ValueError:
                        print("Por favor, insira um número válido para a quantidade.")
                produtos_vendidos[nome] = quantidade
            vendas_realizadas += realizar_venda(produtos_cadastrados, **produtos_vendidos)
        elif escolha == 6:
            gerar_relatorio(vendas_realizadas)
        elif escolha == 7:
            atualizar_csv_estoque(produtos_cadastrados)
            print("Sistema Finalizado.\nObrigado por usar nossos serviços.")
            break
        else:
            print("Opção inválida! Tente novamente.")


main()




    =             Bem-vindo ao             =
    =  Gerenciador de Inventário AdaEletro =
    

    =            Menu Principal            =
    

Lista de Produtos:
ID | Nome | Categoria | Preço | Quantidade | Descrição
1 | Smart TV 55" 4K | Eletrônicos | 299.99 | 50 | Televisão LED 55" com resolução 4K e suporte a HDR.
2 | Smartphone X12 | Eletrônicos | 699.99 | 30 | Smartphone com tela de 6.5", 128GB de armazenamento.
3 | Cadeira de Escritório | Móveis | 89.99 | 15 | Cadeira ergonômica ajustável para escritório.
4 | Notebook UltraBook | Eletrônicos | 1199.99 | 20 | Notebook com processador Intel i7 e 16GB de RAM.
5 | Geladeira Frost Free | Eletrodomésticos | 899.99 | 10 | Geladeira com tecnologia Frost Free e 500L de capacidade.
6 | Mixer de Mão | Eletrodomésticos | 49.99 | 40 | Mixer portátil com 5 velocidades e função turbo.
7 | Tênis Esportivo | Vestuário | 79.99 | 60 | Tênis para corrida com sola de amortecimento.
8 | Relógio de Pulso | Acessórios | 129.99 | 25 | Relógio analóg

KeyboardInterrupt: Interrupted by user